Part 1 - Find 1,4,7,8 by number of wire inputs

In [2]:
import pandas as pd

data = pd.read_csv('test_data.csv', sep=' | ', engine='python', header=None)

def find_1478(data):
    output_values = data.iloc[:,-4:].copy()
    for x in output_values.index.values:
        output_values.iloc[x] = [len(out) for out in output_values.iloc[x].values]
    output_values = output_values.astype('int32')
    logic_df = (output_values == 4) | (output_values == 2) | (output_values == 3) | (output_values == 7)
    return logic_df.values.sum()

find_1478(data)

26

In [3]:
data = pd.read_csv('input_data.csv', sep=' | ', engine='python', header=None)
find_1478(data)

512

Part 2

A little more tricky...each number will have a unique total sum when evaluated. The first example gave us a key to associate the unique total to each number. `determine_cypher` creates a dict object containing unique total values of the number of time each letter appears. For example if 'A' appears 8 times, it is assigned a value of 8. If 'b' appears 7 times it is assigned a values of 7. If the code `ab` appears, it has a unique total value of 17 which is identified as the number 1.

In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('input_data.csv', sep=' | ', engine='python', header=None)

def determine_cypher():
    data = pd.read_csv('test_data.csv', sep=' | ', engine='python', header=None)
    codes = data.iloc[:,:10]
    # Get data input
    row = (
        codes
        .iloc[0]
        .values
        .tolist()
    )
    # Create a df of data with split vals
    splitdf = (
        codes
        .iloc[0]
        .str.zfill(7)
        .str.split('',expand=True)
        .iloc[:,1:8]
        .copy()
    )

    whole_str = ''.join(row)
    letters = ['a','b','c','d','e','f','g']
    counts = create_counts(whole_str, letters)
    for char in letters:
        splitdf[splitdf == char] = counts[char]

    row = pd.DataFrame(row)
    total_vals = splitdf.astype('int32').sum(axis=1)
    encoded = [8,5,2,3,7,9,6,4,0,1]
    row

    cypher = dict().fromkeys([i for i in total_vals],0)
    for idx, key in zip(range(len(total_vals)),cypher.keys()):
        cypher[key] = encoded[idx]
    return cypher

def create_counts(whole_str:str, letters:list):
    dictionary = dict().fromkeys([i for i in letters], 0)
    for char in letters:
        dictionary[char] = whole_str.count(char)
    return dictionary

def decode_wires(data):

    decoded_list = []
    cypher = determine_cypher()
    print(cypher)

    for row_idx in range(len(data)):

        code = data.iloc[row_idx,:10]
        to_decode = data.iloc[row_idx,-4:]
        # Unpack and left fill with zeros into df
        splitdf = (
            code
            .str.zfill(7)
            .str.split('',expand=True)
            .iloc[:,1:8]
            .copy()
        )
        # Cypher creation: counts contains a dict where {letter: count(letter)} in code
        letters = ['a','b','c','d','e','f','g']
        whole_str = ''.join(code.values)
        counts = create_counts(whole_str, letters)

        # Replace the splitdf with letter count before getting sum
        for char in letters:
            splitdf[splitdf == char] = counts[char]

        # Get sum of each code to get unique sum value
        total_vals = splitdf.astype('int32').sum(axis=1)

        # Cypher contains dict with unique total sums to each decoded number
        decoded = [cypher[val] for val in total_vals]
        sorted_strings = [''.join(sorted(x)) for x in code.values]
        final_cypher = dict().fromkeys([i for i in sorted_strings], 0)
        for key, val in zip(final_cypher.keys(), decoded):
            final_cypher[key] = val
        # print(final_cypher)

        # Arrange letters then use as key
        to_decode = [''.join(sorted(x)) for x in to_decode.values]
        decoded_numbers = [final_cypher[key] for key in to_decode]
        string_ints = [str(int) for int in decoded_numbers]
        decoded_list.append(int(''.join(string_ints)))

    return decoded_list

sum(decode_wires(data))

{49: 8, 37: 5, 34: 2, 39: 3, 25: 7, 45: 9, 41: 6, 30: 4, 42: 0, 17: 1}


1091165